In [1]:
import folium
from folium.plugins import MarkerCluster
from folium.plugins import Draw
from folium import plugins
from folium.plugins import MeasureControl
import pandas as pd
import branca
import numpy as np
import vincent
import os
import json
import matplotlib.pyplot as plt
#shows the current version of the library
print(folium.__version__)
print(pd.__version__)
print(np.__version__)
print("Librerias listas")


0.6.0
0.23.4
1.14.3
Librerias listas


crs ( str ,predetermined 'EPSG3857' ) - Define coordinate reference system for proyecting geographic points in coordinates pixel (screen) and vice versa. You can use the Leaflet valors: *EPSG3857: the CRS most common for maps in line, used by almost all commercial and free tile suppliers. Use the spherical mercator projection. Set by default to the crs option on the map. *EPSG4326: A common CRS among the GIS enthusiasts. Use simple equirectangular projection. *EPSG3395: Rarely used by some commercial tile suppliers. Uses the mercator elliptical projection. *Simple: a simple CRS that maps longitude and latitude in X and Y directly. Can be used in flat maps (for example, game maps). Keep in mind that the Y axis must still be reversed (going from bottom to top)

In [ ]:
#attr it's information related to the map that is visualized below
attr = ('&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> '
        'UTEM Cartografia, &copy; <a href="https://github.com/camiloceacarvajal?tab=repositories/attributions">CamiloGithub</a>')
#attr2 is an access with my name that direct me to facebook
attr2 = ('&copy; <a href="https://earth.google.com/web/">google</a> '
        'UTEM Cartografia, &copy; <a href="https://github.com/camiloceacarvajal?tab=repositories/attributions">CamiloGithub</a>')
#define where information is extracted from
cementerio_data = pd.read_csv("Tablacementerio.csv")
#define the name of the map and its initial characteristics such as: scale, zomm, initial coordinates
m = folium.Map(cementerio_data[['Latitudes','Longitudes']].mean().tolist(),
               attr=None,tiles = None, zoom_start=18,control_scale=True,prefer_canvas=True,detect_retina = True)
#open the geojson file with the information and show it on our map
with open(os.path.join('data', 'export.geojson')) as f:
    bars = json.loads(f.read())
plugins.Search(bars, search_zoom=18).add_to(m)
#thunderforest base map
folium.raster_layers.TileLayer(
    tiles='https://tile.thunderforest.com/transport/{z}/{x}/{y}.png?apikey=3cd85f11f4744c0c8c3bdaab8483cde0',
    attr= attr,name='Mapa Base OSM',
    max_zoom=21,subdomains=['mt0', 'mt1', 'mt2', 'mt3'],
    overlay=False,control=True,
).add_to(m)
#googe base tesela
folium.raster_layers.TileLayer(
    tiles='http://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr= attr2,name='Google Earth',
    max_zoom=21,subdomains=['mt0', 'mt1', 'mt2', 'mt3'],
    overlay=False,control=True,
).add_to(m)
#maker that directs to the map of recoleta created by the company esri chile
ref_html = r'<a href="http://munirecoleta.maps.arcgis.com/apps/webappviewer/index.html?id=7361880dbdcd44d1b02ab91c1d07d5eb" target="_blank"> Mapa Recoleta </a>'
pop_html = folium.Popup(folium.Html(ref_html, script=True))
folium.Marker([-33.4177, -70.6504],popup = pop_html,icon=folium.Icon(color='red',icon='tree-deciduous')).add_to(m)
#maker that to the map of mapillary of the cemetery
ref_html = r'<a href="https://www.mapillary.com/app/?lat=-33.41700338888893&lng=-70.65034652777774&z=17&pKey=aTS8sJ-LHWhn5A6Ti5u8rw" target="_blank">Mapa Foto </a>'
pop_html = folium.Popup(folium.Html(ref_html, script=True))
folium.Marker([-33.4178, -70.6497],popup = pop_html,icon=folium.Icon(color='red',icon='tree-deciduous')).add_to(m)
#add tool to the upper right corner
m.add_child(MeasureControl())
#plugins.ScrollZoomToggler().add_to(m)
plugins.Fullscreen(
    position='topright',title='Expandir',
    title_cancel='salir',force_separate_button=True).add_to(m)
#definition and reading of attribute tables
cementerio_data = pd.read_csv("Tablacementerio.csv")
mc = MarkerCluster(name="Patios")
#cycle to generate markers with a visual as excel
for N,data in cementerio_data.iterrows():
    station_html = folium.Html('<b>%s</b>' %(pd.DataFrame(data).to_html()),script=True)
    mc.add_child(folium.Marker([data.Latitudes,data.Longitudes],
 popup = folium.Popup(station_html),icon=folium.Icon(color='green'if data.Status == 'Bueno'
                  else 'red',prefix = 'glyphicon',icon='tree-deciduous')                              
                    ))
#layer controller 
m.add_child(mc)
m.add_child(folium.LayerControl())
#mark latitude longitude of the point or place
m.add_child(folium.LatLngPopup())
#save file with a name
m.save(os.path.join('results', 'Mapacementerio0.91.html'))
#define figure
fig = plt.figure(figsize=(30,10))
plt.subplot2grid((2,3),(0,2))
#indicate date and table
cementerio_data .Status.value_counts(normalize= True).plot(kind="bar",alpha=0.5)
#ttitle of the tables
plt.title("Estado-arbolesPorcentaje")
plt.subplot2grid((2,3),(0,1))
cementerio_data .Status.value_counts().plot(kind='bar',alpha=0.5)
plt.title("Estado-Arbol-total")
plt.ylabel('Datos')
plt.savefig("Ejemplo2.jpg")
#visualization of graphics
fig = plt.figure(figsize=(30,10))
plt.subplot2grid((2,3),(0,2))
cementerio_data .Nombre.value_counts(normalize= True).plot(kind="bar",alpha=0.5)
plt.title("Cantidad de arbol segun especie-porcentaje")
plt.subplot2grid((2,3),(0,1))
cementerio_data .Nombre.value_counts().plot(kind='bar',alpha=0.5)
plt.title("Cantidad de especie en total")
plt.ylabel('Datos')
m